On Wednesday, 6th March, Chancellor of the Exchequer Jeremy Hunt will present the government's 2024 Spring Budget. Ahead of the announcement, media reports speculate that the Chancellor is considering cutting the main National Insurance rate, in an extension to his previous cuts in the Autumn Statement 2023 (analysed by PolicyEngine in [this report](https://policyengine.org/uk/research/autumn-statement-2023)).

In this post, we'll estimate the impacts of such a cut upon the budget, as well as upon society-wide economic outcomes. We'll also look at the impact these changes would have upon individual UK households.

## Budget reform options

Separate media sources identify four potential reforms that the Chancellor is considering:

* Cuts to Employee (Class 1) National Insurance Contributions (NICs) by either 1p[^1] or 2p[^2]
* A 2p cut to the basic rate of tax, lowering the rate from 20p to 18p[^3]

PolicyEngine's top-level economic impact estimates follow below. In modelling these reforms, we assume each took effect on 1 January 2024 and will evaluate their impacts over calendar year 2024.

[^1]:
      [The Guardian, 2024](https://www.theguardian.com/politics/2024/mar/02/tax-and-spending-cuts-will-backfire-economists-warn-jeremy-hunt)
[^2]:
      [Financial Times, 2024](https://www.ft.com/content/c2d56d45-c061-4d5a-bf4f-1eddd1fa7d25)
[^3]:
      [Financial Times, 2024](https://www.ft.com/content/1b4f19d7-c7e5-4d16-a7c1-1c12ec0d0592)

*Table 1: Potential budget reforms and their impacts*

In [49]:
from policyengine_uk.model_api import *
from policyengine_uk import Microsimulation
import pandas as pd

# DataFrame with columns: [provision name, provision cost in 2024, percent better off, provision poverty impact in 2024, inequality impact in 2024]

reforms = []

ni_rate_cut_1p = {
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.09,
    }
}

ni_rate_cut_2p = {
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.08,
    }
}

base_rate_cut_2p = {
    "gov.hmrc.income_tax.rates.uk[0].rate": {
        "year:2024:5": 0.18,
    }
}

reforms = [ni_rate_cut_1p, ni_rate_cut_2p, base_rate_cut_2p]

provisions = ["1p NI cut", "2p NI cut", "2p basic rate cut"]

baseline = Microsimulation()

def get_impacts(reform_dict: dict) -> dict:
    reform = Reform.from_dict(reform_dict, country_id="uk")
    reformed = Microsimulation(reform=reform)

    budgetary_impact = baseline.calculate("household_net_income", period=2024).sum() - reformed.calculate("household_net_income", 2024).sum()
    poverty_baseline = baseline.calculate("in_poverty", map_to="person", period=2024).mean()
    poverty_reform = reformed.calculate("in_poverty", map_to="person", period=2024).mean()
    poverty_rate_change = (poverty_reform - poverty_baseline) / poverty_baseline

    gini_baseline = baseline.calculate("equiv_household_net_income", map_to="person", period=2024).gini()
    gini_reform = reformed.calculate("equiv_household_net_income", map_to="person", period=2024).gini()
    gini_rate_change = (gini_reform - gini_baseline) / gini_baseline

    hnet_reformed = reformed.calculate("household_net_income", period=2024, map_to="person")
    hnet_baseline = baseline.calculate("household_net_income", period=2024, map_to="person")
    percent_better_off = (hnet_reformed > hnet_baseline).mean()
    # Poverty headcount relative change
    # Inequality gini index relative change
    # Percent better off

    return dict(
        budgetary_impact=budgetary_impact,
        reform_id=reform.api_id,
        poverty_rate_change=poverty_rate_change,
        gini_rate_change=gini_rate_change,
        percent_better_off=percent_better_off
    )

budgetary_impacts = []
reform_ids = []
poverty_rate_changes = []
gini_rate_changes = []
percentages_better_off = []

for i in range(len(provisions)):
    impacts = get_impacts(reforms[i])
    budgetary_impacts.append(impacts["budgetary_impact"])
    reform_ids.append(impacts["reform_id"])
    poverty_rate_changes.append(impacts["poverty_rate_change"])
    gini_rate_changes.append(impacts["gini_rate_change"])
    percentages_better_off.append(impacts["percent_better_off"])

df = pd.DataFrame({
    "Reform": provisions,
    "Budgetary impact": budgetary_impacts,
    "Percent gaining": percentages_better_off,
    "Poverty change": poverty_rate_changes,
    "Inequality change": gini_rate_changes,
    "PolicyEngine": reform_ids,
})

df["Budgetary impact"] = df["Budgetary impact"].apply(lambda x: f"{x/1e9:+,.1f}")
df["PolicyEngine"] = df["PolicyEngine"].apply(lambda x: f"[#{x}](https://policyengine.org/uk/policy?reform={x}&baseline=1&time_period=2024&region=uk)")
df["Percent gaining"] =  df["Percent gaining"].apply(lambda x: f"{x:+.1%}")
df["Poverty change"] =  df["Poverty change"].apply(lambda x: f"{x:+.1%}")
df["Inequality change"] =  df["Inequality change"].apply(lambda x: f"{x:+.1%}")

In [44]:
from IPython.display import Markdown

Markdown(df.to_markdown(index=False))

| Reform            |   Budgetary impact | Percent gaining   | Poverty change   | Inequality change   | PolicyEngine                                                                                    |
|:------------------|-------------------:|:------------------|:-----------------|:--------------------|:------------------------------------------------------------------------------------------------|
| 1p NI cut         |               -5   | +63.8%            | +0.0%            | +0.2%               | [#49642](https://policyengine.org/uk/policy?reform=49642&baseline=1&time_period=2024&region=uk) |
| 2p NI cut         |              -10   | +63.8%            | -0.3%            | +0.3%               | [#49643](https://policyengine.org/uk/policy?reform=49643&baseline=1&time_period=2024&region=uk) |
| 2p basic rate cut |              -11.1 | +69.8%            | -0.4%            | +0.3%               | [#49644](https://policyengine.org/uk/policy?reform=49644&baseline=1&time_period=2024&region=uk) |

## Distributional impacts

In [45]:
from policyengine_core.charts.api import get_api_chart_data
import plotly.express as px
from policyengine_core.charts import *

decile_df = pd.DataFrame()

for i in range(len(provisions)):
    impacts = get_impacts(reforms[i])
    decile_data = get_api_chart_data("uk", Reform.from_dict(reforms[i], "uk").api_id, "decile", "uk", 2024, 1)["relative"]
    for decile in range(1, 11):
        decile_df = decile_df.append(
            {
            "Reform": provisions[i],
            "Decile": decile,
            "Relative change": decile_data[str(decile)]
        }, ignoreIndex=True)

fig = px.bar(
    decile_df,
    x="Decile",
    y="Relative change",
    color="Reform",
    barmode="group",
).update_layout(
    title="Change in net income by decile",
    xaxis_title="Reform",
    yaxis_title="Relative change in net income",
    yaxis_tickformat="+.1%",
)

fig = format_fig(fig)
fig

AttributeError: 'DataFrame' object has no attribute 'append'

## National Insurance contributions, 2p decrease

Under this policy, the government would reduce required contributions to the National Insurance system (Class 1 contributions) for individuals employed by an employer. The current rate of 10p, brought into force on 6 January 2024, would be further lowered to 8p.

The PolicyEngine model projects that a reduction in the Class 1 NIC contribution rate by 2p costs the government approximately £10.0 billion. Under this cut, every income decile gains in aggregate, with 63.8% of households increasing their post-tax income, and none seeing a decrease in disposable. This reform would increase inequality, as measured by the Gini coefficient, by 0.3%, while reducing the poverty rate by 0.2%.

For a married couple with two children making £35,000 a year, this reform would lower National Insurance payments by £214 for 2024. However, their maximum possible Universal Credit benefit also falls by £118, providing a net benefit of £96 for the year.

# National Insurance contributions, 1p decrease

Similar to the 2p National Insurance rate cut, under this reform, the government would reduce Class 1 National Insurance contributions by 1p, from the current rate of 10p to 9p.

PolicyEngine projects that this cut would cost the government £5.0 billion. Similar to the 2p cut, under this reform, post-tax income increases for every income decile, with the same 63.8% of households gaining disposable income. This reform would slightly increase the Gini coefficient, by 0.2%, while having a negiligible impact upon the poverty rate.

For the same hypothetical couple from above, this reform would lower National Insurance payments by £115 for the year. However, this family's maximum Universal Credit benefit would fall by a corresponding £63, providing an overall benefit of £52 for 2024.

# Basic Rate decrease

Under another speculated reform, the government would lower the Basic Rate, the tax rate of the UK's lowest tax bracket, by 2p, from its current 20p to 18p. 

As the Basic Rate taxes a larger pool of household income sources than the National Insurance system, PolicyEngine estimates the cost of this reform to be £11.1 billion for 2024. This policy would increase the net income of 69.6% of UK households, with every individual decile also experiencing an increase in net income. Households in higher deciles do particularly well under this reform; households in the UK's top decile would receive £1,175 in extra net income in 2024, while households in the country's lowest decile would receive £16. This policy would lower the poverty rate by 0.4% relative to its current value. However, it would raise the Gini coefficient by 0.3%.

For a two-child household earning £35,000 gross income in 2024, this reform would lower income tax by £197. However, this family's maximum Universal Credit benefit would also decrease by £108, generating a net increase in post-tax income of £89.